In [1]:
import socket
import threading
import time 
from datetime import datetime 
class Tello:
    def __init__(self):
        local_addr = ('', 8889)
        self.tello_addr = ('192.168.10.1', 8889) 
        self.responded = False
        self.log = []
        self.MAX_TIME_OUT = 15.0
        
        self.socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)  # socket for sending cmd
        self.socket.bind(local_addr)
        # background thread for receiving cmd ack
        self.receive_thread = threading.Thread(target=self.receiver)
        self.receive_thread.daemon = True
        self.receive_thread.start()

    def receiver(self): 
        while True:
            try:
                response, ip = self.socket.recvfrom(1024)  
                self.responded = True
                self.log.append( (datetime.now().isoformat(), response) ) 
            except socket.error as exc:
                self.log.append( (datetime.now().isoformat(), f"Caught exception socket.error : {exc}") )  
 
    def send_command(self, command):
        """
        Send a command to tello and do the while loop until background thread changes the state of self.responded, 
        which means a cmd ack is received. Will be blocked until 
        """ 

        self.log.append( (datetime.now().isoformat(), f'Sending command: {command} to {self.tello_addr}') )   
        self.socket.sendto(command.encode('utf-8'), self.tello_addr)

        start = time.time()
        while not self.responded: 
            if time.time() - start > self.MAX_TIME_OUT:
                self.log.append( (datetime.now().isoformat(), f'Max timeout exceeded... command {command}') )  
                return
        self.log.append( (datetime.now().isoformat(), f'Command {command} executed successfully!') )
        print(datetime.now().isoformat() + f': Command {command} executed successfully!')

In [2]:
tello = Tello()

tello.send_command('command')
tello.send_command('takeoff') 
time.sleep(3) 
tello.send_command('land') 

2019-08-15T02:28:08.754791: Command command executed successfully!
2019-08-15T02:28:08.755531: Command takeoff executed successfully!
2019-08-15T02:28:11.760074: Command land executed successfully!


In [3]:
tello.log

[('2019-08-15T02:28:08.739937',
  "Sending command: command to ('192.168.10.1', 8889)"),
 ('2019-08-15T02:28:08.754758', b'ok'),
 ('2019-08-15T02:28:08.754783', 'Command command executed successfully!'),
 ('2019-08-15T02:28:08.755404',
  "Sending command: takeoff to ('192.168.10.1', 8889)"),
 ('2019-08-15T02:28:08.755524', 'Command takeoff executed successfully!'),
 ('2019-08-15T02:28:09.668275', b'ok'),
 ('2019-08-15T02:28:11.759640',
  "Sending command: land to ('192.168.10.1', 8889)"),
 ('2019-08-15T02:28:11.760046', 'Command land executed successfully!')]